# Libs

In [1]:
import os, glob, sys
while os.getcwd().split('\\')[-1] != 'master-llm-rag-vnlaw':
    os.chdir('..')
    print(os.getcwd())

import pandas as pd
import numpy as np
import regex as re
import json
import random


d:\DANG NHI\repos\master-llm-rag-vnlaw


In [2]:
MAX_LEN = 512
OVERLAP = 50

data_path = "data/"
base_model = "FacebookAI/xlm-roberta-base"

files = os.listdir(data_path)
data_stored = []

In [3]:
import torch
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model)

In [4]:
# khi split chunking không quá quan tâm đến độ dài
# mỗi chunk phải là một đoạn hoặc một câu hoàn chỉnh
# nếu là bảng thì phải tách riêng

def preprocess_phrase1_chunking(text):
    s = text
    # s = re.sub('CỘNG HÒA XÃ HỘI CHỦ NGHĨA VIỆT NAM', '', s)
    # s = re.sub('Độc lập - Tự do - Hạnh phúc', '', s)
    s = re.sub('[- ]+', ' ', s)
    s = re.sub('[…]+', '…', s)
    s = s.strip()
    return s

def count_tokens(text):
    return len(tokenizer.encode(text))

def last_words(text, n=10):
    lst = text.split(" ")[-n:]
    return " ".join(lst)

# Simple chunking

In [5]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [6]:
text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size=MAX_LEN,
    chunk_overlap=OVERLAP,
    length_function=count_tokens,
    is_separator_regex=False,
    separators=[
        "\n\n",
        "\n",
        " ",
        ".",
        ",",
        "\u200b",  # Zero-width space
        "\uff0c",  # Fullwidth comma
        "\u3001",  # Ideographic comma
        "\uff0e",  # Fullwidth full stop
        "\u3002",  # Ideographic full stop
        "",
    ]
)

### crafting

In [8]:
for file in files:
    print(file)
    with open(data_path + file, 'r', encoding='utf-8') as f:
        data = json.load(f)
    for i, item in enumerate(data):
        name = item['name']
        print((name, len(item['content'])))
        content = item['content']
        break
    break

BaoHiem.json
('Nghị quyết 06/2024/NQ-HĐND', 65)


In [9]:
content = "\n".join(content)

In [10]:
chunks = text_splitter.split_text(content)

In [11]:
print(chunks[0])

HỘI ĐỒNG NHÂN DÂN TỈNH KHÁNH HÒA 
 Số: 06/2024/NQ-HĐND 
 Khánh Hòa, ngày 12 tháng 7 năm 2024
NGHỊ QUYẾT
VỀ VIỆC BAN HÀNH MỨC GIÁ DỊCH VỤ KHÁM BỆNH, CHỮA BỆNH KHÔNG THUỘC PHẠM VI THANH TOÁN CỦA QUỸ BẢO HIỂM Y TẾ MÀ KHÔNG PHẢI LÀ DỊCH VỤ KHÁM BỆNH, CHỮA BỆNH THEO YÊU CẦU TẠI CÁC CƠ SỞ KHÁM BỆNH, CHỮA BỆNH CỦA NHÀ NƯỚC TRÊN ĐỊA BÀN TỈNH KHÁNH HÒA
HỘI ĐỒNG NHÂN DÂN TỈNH KHÁNH HÒA KHÓA VII, KỲ HỌP THỨ 14
Căn cứ Luật Tổ chức chính quyền địa phương ngày 19 tháng 6 năm 2015;
Căn cứ Luật sửa đổi, bổ sung một số điều của Luật Tổ chức Chính phủ và Luật tổ chức chính quyền địa phương ngày 22 tháng 11 năm 2019;
Căn cứ Luật Ban hành văn bản quy phạm pháp luật ngày 22 tháng 06 năm 2015 ;
Căn cứ Luật sửa đổi, bổ sung một số điều của Luật Ban hành văn bản quy phạm pháp luật ngày 18 tháng 6 năm 2020;
Căn cứ Luật Khám bệnh, chữa bệnh ngày 09 tháng 01 năm 2023;
Căn cứ Luật Giá ngày 20 tháng 06 năm 2012 ;
Căn cứ Nghị định số 177/2013/NĐ-CP ngày 14 tháng 11 năm 2013 của Chính phủ quy định chi tiết và hướng 

In [12]:
print(chunks[1])

Căn cứ Thông tư số 21/2023/TT-BYT ngày 17 tháng 11 năm 2023 của Bộ trưởng Bộ Y tế quy định khung giá dịch vụ khám bệnh, chữa bệnh của Nhà nước và hướng dẫn áp dụng giá, thanh toán chi phí khám bệnh, chữa bệnh trong một số trường hợp;
Xét Tờ trình số 5582/TTr-UBND ngày 27 tháng 5 năm 2024 của Ủy ban nhân dân tỉnh; Báo cáo thẩm tra số 92/BC-BHXH ngày 04 tháng 7 năm 2024 của Ban Văn hóa - Xã hội Hội đồng nhân dân tỉnh; tiếp thu giải trình của Ủy ban nhân dân tỉnh tại Báo cáo số 216/BC-UBND ngày 08 tháng 7 năm 2024 và ý kiến thảo luận của đại biểu Hội đồng nhân dân tại kỳ họp.
QUYẾT NGHỊ:
Điều 1. Ban hành kèm theo Nghị quyết này quy định mức giá dịch vụ khám bệnh, chữa bệnh không thuộc phạm vi thanh toán của Quỹ bảo hiểm y tế mà không phải là dịch vụ khám bệnh, chữa bệnh theo yêu cầu tại các cơ sở khám bệnh, chữa bệnh của Nhà nước trên địa bàn tỉnh Khánh Hòa như sau:
1. Phạm vi điều chỉnh và đối tượng áp dụng
a) Phạm vi điều chỉnh: Nghị quyết này quy định mức giá dịch vụ khám bệnh, chữa bệ

In [13]:
print(chunks[2])

a) Giá dịch vụ khám bệnh, kiểm tra sức khỏe (tại Phụ lục I kèm theo);
b) Giá dịch vụ ngày giường điều trị (tại Phụ lục II kèm theo);
c) Giá dịch vụ kỹ thuật, xét nghiệm (tại Phụ lục III kèm theo);
Điều 2. Tổ chức thực hiện
1. Giao Ủy ban nhân dân tỉnh tổ chức triển khai thực hiện Nghị quyết này theo đúng quy định của pháp luật.
2. Giao Thường trực Hội đồng nhân dân tỉnh, các Ban Hội đồng nhân dân tỉnh, các Tổ đại biểu Hội đồng nhân dân tỉnh và các đại biểu Hội đồng nhân dân tỉnh giám sát việc thực hiện Nghị quyết này.
3. Nghị quyết này thay thế Nghị quyết số 09/2019/NQ-HĐND ngày 11 tháng 12 năm 2019 của Hội đồng nhân dân tỉnh Khánh Hòa về việc ban hành mức giá dịch vụ khám bệnh, chữa bệnh không thuộc phạm vi thanh toán của Quỹ bảo hiểm y tế trong các cơ sở khám bệnh, chữa bệnh của Nhà nước trên địa bàn tỉnh Khánh Hòa.
Nghị quyết này đã được Hội đồng nhân dân tỉnh Khánh Hoà Khoá VII, Kỳ họp thứ 14 thông qua ngày 12 tháng 7 năm 2024 và có hiệu lực từ ngày 22 tháng 7 năm 2024.
Điều 3. Quy

### final run all

In [8]:
data_stored = []

for file in files:
    print(file)
    field = []

    with open(data_path + file, 'r', encoding='utf-8') as f:
        data = json.load(f)

    for i, item in enumerate(data):
        name = item['name']
        print((name, len(item['content'])))
        content = item['content']
        content = "\n".join(content)
        chunks = text_splitter.split_text(content)
        field.append(chunks)
        if i == 10:
            break
        
    data_stored.append(field)

BaoHiem.json
('Nghị quyết 06/2024/NQ-HĐND', 65)
('Nghị quyết 05/2024/NQ-HĐND', 22)
('Nghị quyết 26/2024/NQ-HĐND', 68)


Token indices sequence length is longer than the specified maximum sequence length for this model (958 > 512). Running this sequence through the model will result in indexing errors


('Văn bản hợp nhất 3305/VBHN-BLĐTBXH', 934)
('Công văn 5652/BHXH-KHTC', 11)
('Công văn 2646/BHXH-TCKT', 19)
('Công văn 2699/BHXH-VP', 12)
('2840/QCPH-BHXH-HNCT', 73)
('Thông báo 703/TB-BHXH', 7)
('Quyết định 1603/QĐ-BHXH', 226)
('Nghị quyết 02/2024/NQ-HĐND', 45)
BoMayHanhChinh.json
('Quyết định 770/QĐ-UBND', 25)
('Quyết định 1017/QĐ-UBND', 21)
('Kế hoạch 91/KH-UBND', 57)
('Quyết định 774/QĐ-UBND', 24)
('Quyết định 519/QĐ-UBND', 21)
('Quyết định 1016/QĐ-UBND', 21)
('Quyết định 533/QĐ-BGTVT', 116)
('Quyết định 856/QĐ-BKHCN', 70)
('Quyết định 09/2024/QĐ-UBND', 30)
('Công văn 947/QLD-ĐK', 10)
('Quyết định 527/QĐ-UBND', 27)
CoCauToChuc.json
('Quyết định 08/2023/QĐ-UBND', 175)
('Nghị định 11/2023/NĐ-CP', 113)
('Quyết định 20/2023/QĐ-UBND', 152)
('Quyết định 06/2023/QĐ-UBND', 158)
('Quyết định 09/2023/QĐ-UBND', 16)
('Kế hoạch 71/KH-UBND', 47)
('Quyết định 1586/QĐ-BNN-TCCB', 112)
('Quyết định 1589/QĐ-BNN-TCCB', 108)
('Quyết định 1588/QĐ-BNN-TCCB', 138)
('Quyết định 16/2023/QĐ-UBND', 95)
('Quyế

TypeError: list indices must be integers or slices, not str

# Pairing

In [ ]:
def get_very_positive_pairs_chunks(data_stored, max_pairs= 500000):
    pairs = []
    for i in range(len(data_stored)):
        for j in range(len(data_stored[i])):
            chunks = data_stored[i][j]
            for k in range(len(chunks)):
                for l in range(k+1, len(chunks)):
                    pairs.append(((i,j,k), (i,j,l), 0.98))
    return random.sample(pairs, k=min(max_pairs, len(pairs)))

def get_low_positive_pairs_chunks(data_stored):
    pairs = []
    for i in range(len(data_stored)):
        l = len(data_stored[i])
        for j in range(l):
            for k in range(j+1, l):
                a = random.randrange(len(data_stored[i][j]))
                b = random.randrange(len(data_stored[i][k]))
                pairs.append(((i, j, a) , (i,k,b), 0.6))
    return pairs

def get_negative_pairs(data_stored, mean_per_field_pair=6746):
    pairs = []
    l = len(data_stored)
    for i in range(l):
        for j in range(i+1, l):
            indices1 = [(i, a, b) for a in range(len(data_stored[i])) for b in range(len(data_stored[i][a]))]
            indices2 = [(j, a, b) for a in range(len(data_stored[j])) for b in range(len(data_stored[j][a]))]

            a = np.random.choice(len(indices1), mean_per_field_pair)
            b = np.random.choice(len(indices2), mean_per_field_pair)

            for x, y in zip(a, b):
                pairs.append((indices1[x], indices2[y], 0))
    return pairs

In [12]:
pairs_doc = 0
for i in range(len(data_stored)):
    l = len(data_stored[i])
    pairs_doc += (l*(l-1))//2
pairs_doc

330

In [13]:
very_positive_pairs = get_very_positive_pairs_chunks(data_stored)
len(very_positive_pairs)

157547

In [14]:
low_positive_pairs = get_low_positive_pairs_chunks(data_stored)
len(low_positive_pairs)

330

In [15]:
l = len(data_stored)
pairs_field = (l*(l-1))//2

mean_per_field_pair = len(low_positive_pairs) // pairs_field
mean_per_field_pair

22

In [17]:
negative_pairs = get_negative_pairs(data_stored, mean_per_field_pair)
len(negative_pairs)

330

# Training

In [18]:
from tqdm.auto import tqdm
from transformers import get_scheduler
from transformers import AutoModel
model = AutoModel.from_pretrained(base_model)

Some weights of the model checkpoint at FacebookAI/xlm-roberta-base were not used when initializing XLMRobertaModel: ['lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.bias']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [23]:
from torch.utils.data import DataLoader, Dataset

class CustomDataset(Dataset):
    def __init__(self, pairs):
        self.pairs = pairs
    def __len__(self):
        return len(self.pairs)
    def __getitem__(self, idx):
        pair = self.pairs[idx]
        (i,j,k), (x,y,z), score = self.pairs[idx]
        sentence1 = data_stored[i][j][k]
        sentence2 = data_stored[x][y][z]

        token1 = tokenizer(sentence1, truncation=True, padding='max_length', max_length=MAX_LEN, return_tensors='pt')
        token2 = tokenizer(sentence2, truncation=True, padding='max_length', max_length=MAX_LEN, return_tensors='pt')

        return token1, token2, score

In [ ]:
dataset = CustomDataset(very_positive_pairs + low_positive_pairs + negative_pairs)
dataloader = DataLoader(dataset, batch_size=10, shuffle=True)

In [30]:
for batch in dataloader:
    token1, token2, score = batch
    break

In [33]:
token1['attention_mask'].shape

torch.Size([100, 1, 512])

In [ ]:
import torch
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
# loss_function = torch.nn.CosineEmbeddingLoss()
loss_function = torch.nn.MSELoss()

In [ ]:
for batch in dataloader:
    token1, token2, score = batch
    # outputs1 = model(token1['input_ids'].squeeze(1).to(device), token1['attention_mask'].squeeze(1).to(device))
    # outputs2 = model(token2['input_ids'].squeeze(1).to(device), token2['attention_mask'].squeeze(1).to(device))
    outputs1 = model(token1['input_ids'].squeeze(1).to(device))
    outputs2 = model(token2['input_ids'].squeeze(1).to(device))
    similarity = torch.cosine_similarity(outputs1.pooler_output, outputs2.pooler_output)
    loss = loss_function(similarity, score.float().to(device))

    print(loss.item(), score.numpy())
    loss.backward()
    optimizer.step()
